### User Input

In [ ]:
pruning_user_params = {
    'model_depth': 3,        # 3, 5, 7
    'prune layers': 'dense', # none, all, dense, or conv (does not work right now)
    'verbose_level': 0
}

### Path Setup

In [ ]:
depth = pruning_user_params['model_depth']
verbosity = pruning_user_params['verbose_level']

mfccs_json_path = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale_emptyocean.json"
saved_model_path = f'../saved_model/layers{depth}/'
pruned_model_path = f'../saved_model/pruned_models/layers{depth}/'
parameter_csv_path = f'../model-stats/layers-{depth}_filters-1-16_n-trail1.csv'
plot_file_name = f'../images/layers-{depth}_baseline-and-pruned-model-size-vs.png'

### Imports

In [ ]:
import os
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import matplotlib.pyplot as plt

### Load in the MFCC File

In [ ]:
from compression_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

### Prepare the Datasets

In [ ]:
# create train, validation and test sets
from compression_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.6, 0.5) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Load in Saved Models

In [ ]:
model_list = []
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(saved_model_path)):
    for f in filenames:
        loaded_model = tf.keras.models.load_model(dirpath+f)
        model_list.append(loaded_model)
print(f'loaded in {len(model_list)} baseline models')

### Determine Baseline Model Accuracies

In [ ]:
model_accuracy = []
for model in model_list:
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    model_accuracy.append(test_accuracy)
print(f'determined {len(model_accuracy)} accuracy values with max. accuracy: {max(model_accuracy)} and min accuracy: {min(model_accuracy)}')

### Establish Pruned Model Settings

In [ ]:
# create a prune_low_magnitude object
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

### Generate List of Pruned Models

In [ ]:
pruned_model_list = []
for model in model_list:
    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    pruned_model_list.append(model_for_pruning)
print(f'created {len(pruned_model_list)} pruned models')

### Train the Pruned Models

In [ ]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

model_history = []
for model in pruned_model_list:
  history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                      validation_split=validation_split, callbacks=callbacks, verbose=verbosity)
  model_history.append(history)

In [ ]:
for history in model_history:
    print (history.history['val_accuracy'])

### Get Accuracy values for Pruned Models

In [ ]:
pruned_model_accuracy = []
for model in pruned_model_list:
   _, model_for_pruning_accuracy = model.evaluate(X_test, y_test, verbose=0)
   pruned_model_accuracy.append(model_for_pruning_accuracy)

In [ ]:
print(pruned_model_accuracy)

### Save the Pruned Models

In [ ]:
for dense_model_name, pruned_model in zip(filenames, pruned_model_list):
    model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)
    pruned_keras_file = pruned_model_path + dense_model_name
    tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)

### Get list of Baseline Model Memory Size

In [ ]:
from compression_lib import get_gzipped_model_size

baseline_model_sizes = []
for f in filenames:
    baseline_model_path = saved_model_path + f
    model_size = get_gzipped_model_size(baseline_model_path)
    baseline_model_sizes.append(model_size)
print(baseline_model_sizes)

### Get List of Pruned Model Memory Size

In [ ]:
from compression_lib import get_gzipped_model_size

pruned_model_sizes = []
for f in filenames:
    model_path = pruned_model_path + f
    model_size = get_gzipped_model_size(model_path)
    pruned_model_sizes.append(model_size)
print(pruned_model_sizes)

### Plot Baseline and Pruned Curves

In [ ]:

plt.plot(baseline_model_sizes, model_accuracy, 'o', color='blue')
plt.plot(pruned_model_sizes, pruned_model_accuracy, 'o', color='red')
plt.xlabel("Number of Bytes")
plt.ylabel("Model Accuracy")
plt.savefig(plot_file_name)

In [ ]:
import pandas as pd  
aa = pd.read_csv(parameter_csv_path)  
aa["Baseline Size"] = baseline_model_sizes[:16]
aa["Pruned Accuracy"] = pruned_model_accuracy[:16]
aa["Pruned Size"] = pruned_model_sizes[:16]
aa.to_csv(parameter_csv_path)